In [ ]:
df = pd.read_json("../database-unclean/all/train_period_data.jsonlist", orient='list', lines=True)


In [39]:
def grab_top_comment(post_id):
   submission = reddit.submission(id=post_id)
   submission.submission_type = 'best'
   submission.comments.replace_more(limit=0)
   comments = list(submission.comments)[0].replies.list()
   non_op_responses = []
   for c in comments:
      # if c.author is not None:
      #    if c.author.name == submission.author.name:
      #       #print("boop")
      #       non_op_responses.append("OP: " + c.body)
      # else:
      #    non_op_responses.append(c.body)
      non_op_responses.append(c.body)
   return non_op_responses

In [40]:
comments = grab_top_comment("2r3lm3")
comments

["> But if you're shooting heroin, there's a good chance you'll become hooked.\n\nI get what you're saying but this a common misconception that I hate seeing repeated again and again. According to government stats less than a quarter of all heroin users ever become dependent on it. It has a high rate of dependency compared to other drugs, but the vast majority of users never become dependent - even if you assume that that figure is a conservative estimate.\n\n[Source](http://www.drugabuse.gov/publications/drugfacts/heroin)",
 '> Drugs\n\nThis part is interesting because of the description of the complex nature of the situation. In the case of crimes resulting from drugs, I mentioned below that I believe the crime itself should be the primary offence, and the drug use which facilitated it would be a secondary offence that get tagged onto crimes. This would punish those who go crazy from drugs, without inherently punishing anyone for possession or use who has otherwise done nothing wrong

In [41]:
len(comments)

53

In [17]:
from transformers import pipeline
summarizer = pipeline("summarization", model="t5-small")

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

C:\Users\yokon\anaconda3\lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [42]:
print(len(comments))
for comment in comments:
   print(summarizer(comment, min_length=0, max_length=200))

Your max_length is set to 200, but you input_length is only 133. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)


53
[{'summary_text': 'less than a quarter of all heroin users ever become dependent on it . the vast majority of users never become dependent - even if you assume that that figure is a conservative estimate .'}]
[{'summary_text': 'in the case of crimes resulting from drugs, I mentioned that the crime itself should be the primary offence . this would punish those who go crazy from drugs without inherently punishing anyone for possession or use who has otherwise done nothing wrong . the current stigma of "just trying to get high is wrong" marginalizes and generalizes a much more complex issue .'}]
[{'summary_text': "a huge part of what keeps them down is the feeling of being social outcasts . even giving them access to free, clean drugs ends the cycle faster and cheaperly because they aren't exposed to nearly as many risks like sexual abusing themselves, criminal behavior and the risk of OD ."}]


Your max_length is set to 200, but you input_length is only 23. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)


[{'summary_text': "heroin is just a powder extracted from a plant, but it's not inherently problematic . if heroin were not illegal it would cost pennies to feed a habit, meaning an addict would not have to rob or steal . the moral fabric has not degraded to a point where we have been ruined by it ."}]


Your max_length is set to 200, but you input_length is only 50. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)


...

[{'summary_text': 'this argument could be used to justify anything being legal so long as it had no absolutely direct harm to another human being . i believe you should be able to do absolutely anything you want in the world as long as you do not harm others.'}]
[{'summary_text': "we don't ban alcohol because of drunk driving . some drugs also get you intoxicated . the intoxication from some drugs is benign ."}]


Your max_length is set to 200, but you input_length is only 50. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=25)


[{'summary_text': 'only 2% of the people interviewed for that study tried Heroin... and 23% became addicted . alcohol and tobacco are sold cheaply and legally and alcohol is basically a cultural norm at this point . if heroin was as socially accepted as cigarettes and cigarettes, the percentage would go up .'}]
[{'summary_text': 'a bevy of indirect harms in this case has no place . otherwise it could make sense, you can read JSM if you like .'}]


Your max_length is set to 200, but you input_length is only 96. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)


[{'summary_text': 'heroin is currently socially unacceptable and a very uncommon drug . 2% who are doing it now are probably more risk-taking, impulsive, antisocial, less health-conscious, etc than the average person . people with these traits are more likely to become dependent .'}]
[{'summary_text': "we should stick to direct or very close to direct events causing harm . there's no point in debating if we're going to get rendered down to semantics ."}]
